In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

## Import the Library

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import os
from glob import glob
from PIL import Image
from tqdm import tqdm


cudnn.benchmark = True

In [ ]:
def imshow(img, title):
    
    npimg = img.numpy() / 2 + 0.5
    plt.figure(figsize= (batch_size, 1))
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.show()
    
def show_batch_images(dataloder):
    
    images, labels = next(iter(dataloder))
    img = torchvision.utils.make_grid(images)
    imshow(img, title = [ str(x.item()) for x in labels])    

## Code to load images below

In [ ]:
files = glob("/content/gdrive/MyDrive/ISB/data/train/*/*")

min_h = 1024
min_w = 1024
max_h = 0
max_w = 0
count = 0

for file_ in tqdm(files): 
  img = Image.open(file_)
  width, height = img.size

  if count == 0:
    min_h, max_h = height, height
    min_w, max_w = width, width
    count += 1

  if width < min_w:
    min_w = width
  if height < min_h:
    min_h = height
  if max_h < height:
    max_h = height
  if max_w < width:
    max_w = width

In [ ]:
# Total training data
print("Total train data:", len(files))

print(min_w)
print(min_h)
print(max_h)
print(max_w)

img.size

In [ ]:
files = glob("/content/gdrive/MyDrive/ISB/data/val/*/*")

min_h = 1024
min_w = 1024
max_h = 0
max_w = 0
count = 0

for file_ in tqdm(files): 
  img = Image.open(file_)
  width, height = img.size

  if count == 0:
    min_h, max_h = height, height
    min_w, max_w = width, width
    count += 1

  if width < min_w:
    min_w = width
  if height < min_h:
    min_h = height
  if max_h < height:
    max_h = height
  if max_w < width:
    max_w = width

In [ ]:
# Total validation data
print("Total validation data:", len(files))

print(min_w)
print(min_h)
print(max_h)
print(max_w)

img.size

## Applying data augmentation and resizing images to (128, 128)

In [ ]:
data_dir = '/content/gdrive/MyDrive/ISB/data'

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128,128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.RandomGrayscale(p=0.1),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((128,128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.RandomGrayscale(p=0.1),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

## Loading the image dataset

In [ ]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

train_dataloaders = torch.utils.data.DataLoader(image_datasets['train'], batch_size=64,
                                             shuffle=True, num_workers=2)

val_dataloaders = torch.utils.data.DataLoader(image_datasets['val'], batch_size=64,
                                             shuffle=False, num_workers=2)


dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
len(class_names)  # total validation class

## Building the Model

In [ ]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 100)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001, betas=(0.9,0.999))

# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [ ]:
train_losses, test_losses = [], []

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'train':
              train_losses.append(running_loss / dataset_sizes[phase])

            if phase =="val":
              test_losses.append(running_loss / dataset_sizes[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:

                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


    # load best model weights
    model.load_state_dict(best_model_wts)

    plt.title('Performance VS Epochs')
    plt.plot(train_losses, label='Training loss')
    plt.plot(test_losses, label='Validation loss')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(frameon=True)
    plt.show()

    return model

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)